## AWS CUR QUERY LIBRARY

The CUR Query Library is a collection of curated SQL queries to get you started with analyzing your Cost and Usage Report (CUR) data. Cost analysis is unique to each business and these queries are intended to be modified to suit your specific needs. The library will be updated periodically as we build new queries and receive feedback from customers.

#### [Link - AWS CUR QUERY LIBRARY](https://wellarchitectedlabs.com/cost/300_labs/300_cur_queries/)

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import to_date
from pyspark.sql import Row

spark = SparkSession.builder \
      .master("local[1]") \
      .appName("SparkByExamples.com") \
      .getOrCreate()
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.set_option('display.max_columns', 500)

23/01/04 14:48:27 WARN Utils: Your hostname, codespaces-244068 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
23/01/04 14:48:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/04 14:48:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.parquet("data")

# Create year and month columns
df = df.withColumn('year',year(df.line_item_usage_start_date))
df = df.withColumn('month',month(df.line_item_usage_start_date))
df = df.withColumn('day', dayofmonth(df.line_item_usage_start_date))

In [4]:
total_rows = df.count()
print('{:,}'.format(total_rows))

334,911


In [7]:
df.limit(5).toPandas()

,identity_line_item_id,identity_time_interval,bill_invoice_id,bill_billing_entity,bill_bill_type,bill_payer_account_id,bill_billing_period_start_date,bill_billing_period_end_date,line_item_usage_account_id,line_item_line_item_type,line_item_usage_start_date,line_item_usage_end_date,line_item_product_code,line_item_usage_type,line_item_operation,line_item_availability_zone,line_item_resource_id,line_item_usage_amount,line_item_normalization_factor,line_item_normalized_usage_amount,line_item_currency_code,line_item_unblended_rate,line_item_unblended_cost,line_item_blended_rate,line_item_blended_cost,line_item_line_item_description,line_item_tax_type,line_item_legal_entity,product_product_name,product_availability,product_capacitystatus,product_category,product_clock_speed,product_current_generation,product_dedicated_ebs_throughput,product_description,product_durability,product_ecu,product_edition,product_enhanced_networking_supported,product_event_type,product_free_query_types,product_from_location,product_from_location_type,product_group,product_group_description,product_instance_family,product_instance_type,product_instance_type_family,product_license_model,product_location,product_location_type,product_max_iops_burst_performance,product_max_iopsvolume,product_max_throughputvolume,product_max_volume_size,product_memory,product_message_delivery_frequency,product_message_delivery_order,product_network_performance,product_normalization_size_factor,product_operating_system,product_operation,product_physical_processor,product_pre_installed_sw,product_processor_architecture,product_processor_features,product_product_family,product_queue_type,product_region,product_servicecode,product_servicename,product_sku,product_storage,product_storage_class,product_storage_media,product_subscription_type,product_tenancy,product_to_location,product_to_location_type,product_transfer_type,product_usagetype,product_vcpu,product_version,product_volume_type,pricing_lease_contract_length,pricing_offering_class,pricing_purchase_option,pricing_rate_id,pricing_public_on_demand_cost,pricing_public_on_demand_rate,pricing_term,pricing_unit,reservation_amortized_upfront_cost_for_usage,reservation_amortized_upfront_fee_for_billing_period,reservation_effective_cost,reservation_end_time,reservation_modification_status,reservation_normalized_units_per_reservation,reservation_number_of_reservations,reservation_recurring_fee_for_usage,reservation_reservation_a_r_n,reservation_start_time,reservation_subscription_id,reservation_total_reserved_normalized_units,reservation_total_reserved_units,reservation_units_per_reservation,reservation_unused_amortized_upfront_fee_for_billing_period,reservation_unused_normalized_unit_quantity,reservation_unused_quantity,reservation_unused_recurring_fee,reservation_upfront_value,resource_tags_aws_autoscaling_group_name,resource_tags_aws_created_by,resource_tags_aws_ec2spot_fleet_request_id,resource_tags_user_cost_center,resource_tags_user_department,resource_tags_user_environment,resource_tags_user_name,resource_tags_user_workload,year,month,day
0,kis3kg4pvuwftwpgay7swjwgilgqhy3qykq34hb5dj4bvb...,2018-11-01T00:00:00Z/2018-11-30T23:59:59Z,159263274,AWS,Anniversary,836060457634,2018-11-01,2018-12-01,836060457634,RIFee,2018-11-01,2018-11-30 23:59:59,AmazonEC2,HeavyUsage:c4.large,RunInstances,,,720.0,4.0,2880.0,USD,0.0300000000,21.600,0.0300000000,21.600,USD 0.03 hourly fee per Linux/UNIX (Amazon VPC...,,"Amazon Web Services, Inc.",Amazon Elastic Compute Cloud,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,us-east-1,,,Y4Y2JABGGGHB94YA,,,,,,,,,,,,,,,,18526829,0.0,,,,0.0,21.616438,0.0,2019-04-24T00:34:00.000Z,Original,2880.00000000000,1,0.0,arn:aws:ec2:us-east-1:836060457634:reserved-in...,2018-04-24T00:34:01.000Z,1141706146,2880.00000000000,720.0000000000,720.0000000000,21.616438,2880.0,720.0,21.6,263.0,,,,,,,,,2018,11,1
1,rwk76dksjly2ic32vtkswon2kimlfbmld3s4jjp7oxk473...,2018-11-01T00:00:00Z/2018-11-30T23:59:59Z,159263274,AWS,Anniversary,836060

In [24]:
total_products = len(df.select('product_product_name').distinct().toPandas())
print('Total products: {:,}'.format(total_products))

Total products: 17


In [28]:
services = df.select('product_product_name').toPandas()
services.value_counts().rename_axis('unique_values').to_frame('counts')

,counts
unique_values,
Amazon Elastic Compute Cloud,158330
Amazon Simple Storage Service,106871
Amazon GuardDuty,26326
AWS CloudTrail,14508
Amazon Relational Database Service,11929
AWS Glue,6473
Amazon QuickSight,4623
Amazon Simple Notification Service,2925
AmazonCloudWatch,1253


In [8]:
df.createOrReplaceTempView("tableA")

In [9]:
def sql_query(sql_file, table_name, date_filter):
    with open('queries/' + sql_file) as f:
        query = f.read()
    sql = query.replace("${table_name}", table_name) \
                .replace("${table_Name}", table_name) \
                .replace("${date_filter}", date_filter) \
                .replace("%Y", "y") \
                .replace("%m", "M") \
                .replace("%d", "d")
    
    sql_df = spark.sql(sql)
    return sql_df

### Athena Query

In [11]:
# This query will provide the top 20 Amazon Athena workgroups driving cost within an Organization.
# link: https://wellarchitectedlabs.com/cost/300_labs/300_cur_queries/queries/analytics/

table_name = "tableA"
date_filter = """
year = '2018' 
AND month BETWEEN '10' AND '12'
"""

df_sql = sql_query("athena.sql", table_name, date_filter)
df_sql.toPandas().head()

,bill_payer_account_id,line_item_usage_account_id,day_line_item_usage_start_date,line_item_usage_type,line_item_resource_id,product_region,line_item_product_code,sum_line_item_usage_amount,sum_line_item_unblended_cost
0,836060457634,836060457634,2018-10-30,USE1-DataScannedInTB,,us-east-1,AmazonAthena,0.001640,0.00820000
1,836060457634,836060457634,2018-11-13,USE1-DataScannedInTB,,us-east-1,AmazonAthena,0.001060,0.00530000
2,836060457634,836060457634,2018-10-25,USE1-DataScannedInTB,,us-east-1,AmazonAthena,0.000918,0.00459000
3,836060457634,836060457634,2018-11-5,USE1-DataScannedInTB,,us-east-1,AmazonAthena,0.000890,0.00445000
4,836060457634,836060457634,2018-10-29,USE1-DataScannedInTB,,us-east-1,AmazonAthena,0.000620,0.00310000


### S3 Query

In [12]:
table_name = "tableA"
date_filter = """
year = '2018' 
AND month = '12'
"""

df_sql = sql_query("s3.sql", table_name, date_filter)
df_sql.toPandas().head()

,bill_payer_account_id,line_item_usage_account_id,day_line_item_usage_start_date,split_line_item_resource_id,product_product_name,sum_line_item_usage_amount,sum_line_item_unblended_cost
0,836060457634,011254002932,2018-12-1,,Amazon Simple Storage Service,3.000110,0E-8
1,836060457634,011254002932,2018-12-1,,Amazon Simple Storage Service,3.000023,0E-8
2,836060457634,011254002932,2018-12-1,,Amazon Simple Storage Service,3.000049,0E-8
3,836060457634,836060457634,2018-12-1,,Amazon Simple Storage Service,16.000156,0E-8
4,836060457634,011254002932,2018-12-1,,Amazon Simple Storage Service,7.000000,0E-8


In [13]:
table_name = "tableA"
date_filter = """
year = '2018' 
"""

df_sql = sql_query("s3costusagetypewrid.sql", table_name, date_filter)
df_sql.toPandas().head()

,bill_payer_account_id,line_item_usage_account_id,day_line_item_usage_start_date,line_item_resource_id,line_item_operation,case_line_item_usage_type,sum_line_item_usage_amount,sum_line_item_unblended_cost
0,836060457634,836060457634,2018-11-27,costworkshop-billing,ListBucket,Requests Standard,7440.0,0.03720000
1,836060457634,836060457634,2018-11-2,natbesh-httpd-logs,ListBucket,Requests Standard,1980.0,0.00990000
2,836060457634,011254002932,2018-11-19,cloudtrail-awslogs-011254002932-m0er4rrl-iseng...,PutObject,Requests Standard,1168.0,0.00584000
3,836060457634,836060457634,2018-10-10,cloudtrail-awslogs-836060457634-t7dhuihl-iseng...,PutObject,Requests Standard,1146.0,0.00573000
4,836060457634,836060457634,2018-10-9,cloudtrail-awslogs-836060457634-t7dhuihl-iseng...,PutObject,Requests Standard,1127.0,0.00563500


In [14]:
table_name = "tableA"
date_filter = """
year = '2018'
"""

df_sql = sql_query("billservice.sql", table_name, date_filter)
df_sql.toPandas().head()

,month_line_item_usage_start_date,bill_bill_type,case_product_product_name,product_location,line_item_line_item_description,round_sum_line_item_unblended_cost,sum_line_item_usage_amount
0,2018-10-01,Anniversary,Athena,US East (N. Virginia),Data scanned per query in Terabytes,0.02233,0.004466
1,2018-10-01,Anniversary,Budgets,Any,$0.00 for 62 Budget-days per month (free tier),0.00000,62.000000
2,2018-10-01,Anniversary,Budgets,Any,$0.02 per Budget-day,0.24000,12.000000
3,2018-10-01,Anniversary,CloudTrail,,0.0 per free event recorded in Asia Pacific (O...,0.00000,204.000000
4,2018-10-01,Anniversary,CloudTrail,Asia Pacific (Mumbai),0.0 per free event recorded in Asia Pacific (M...,0.00000,912.000000


In [15]:
table_name = "tableA"
date_filter = """
year = '2018'
"""

df_sql = sql_query("ec2ricoverage.sql", table_name, date_filter)
df_sql.toPandas().head()

,bill_payer_account_id,line_item_usage_account_id,day_line_item_usage_start_date,case_purchase_option,split_reservation_reservation_a_r_n,split_line_item_usage_type_instance_type,split_line_item_usage_type_instance_family,case_product_region,line_item_line_item_type,sum_line_item_usage_amount,sum_reservation_unused_quantity,sum_line_item_normalized_usage_amount,sum_reservation_unused_normalized_unit_quantity,sum_reservation_effective_cost,sum_line_item_unblended_cost
0,836060457634,836060457634,2018-10-1,ReservedInstance,3a2944f2-fbdc-4f9a-86f7-4b83dc6e058a,c4.large,c4,us-east-1,RIFee,744.0,744.0,2976.0,2976.0,0E-8,22.32000000
1,836060457634,836060457634,2018-10-1,ReservedInstance,2a64512a-6eea-493f-967b-cd946b6e496d,c5.large,c5,us-east-1,RIFee,1488.0,0.0,5952.0,0.0,0E-8,38.68800000
2,836060457634,836060457634,2018-10-1,OnDemand,,c5.large,c5,us-east-1,Usage,48.0,0.0,192.0,0.0,0E-8,4.08000000
3,836060457634,836060457634,2018-10-1,ReservedInstance,2a64512a-6eea-493f-967b-cd946b6e496d,c5.large,c5,us-east-1,DiscountedUsage,48.0,0.0,192.0,0.0,2.46991776,0E-8
4,836060457634,836060457634,2018-10-1,OnDemand,,m4.xlarge,m4,us-east-1,Usage,24.0,0.0,192.0,0.0,0E-8,4.80000000
